In [ ]:
import logging
import json
import os
import re
from pathlib import Path
from collections import Counter

import anndata
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import tifffile
from einops import rearrange
from skimage.measure import regionprops

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
import multiplex_imaging_pipeline.utils as utils
import multiplex_imaging_pipeline.ome as ome
import multiplex_imaging_pipeline.segmentation as seg
import multiplex_imaging_pipeline.multiplex_imaging_pipeline as mip
from multiplex_imaging_pipeline.spatial_features import DEFAULT_GATING_STRATEGY

In [ ]:
parent_dir = f'/diskmnt/Projects/Users/estorrs/imaging-analysis/results/mushroom'
specimen_metadata_fp = '/diskmnt/Projects/Users/estorrs/sandbox/specimen_tracking.tsv'

## Define inputs

In [ ]:
def qc_channel_names():
    mapping = utils.R_CHANNEL_MAPPING
    for fp in metadata['filepath']:
        channels = utils.get_ome_tiff_channels(fp)
        for c in channels:
            assert c in mapping, f'channel {c} is not in channel mapping, to ensure all channel names match consider adding {c} to channel mapping. If you do not it may impact cell gating.'

        print(fp, [mapping[c] for c in channels])
        
def get_channel_thresholds():
    channel_thresholds = json.load(open(os.path.join(output_dir, 'channel_thresholds.json')))
    channel_thresholds = {k.split(' | ')[-1]:v for k, v in channel_thresholds.items()} # fps should be unique
    return channel_thresholds

###### HT413C1-Th1k4A1

In [ ]:
case = 'HT413C1-Th1k4A1'
runs = [
    '2024-01-11_HT413C1_Th1k4A1',
    '2024-01-10_HT413C1_Th1k4A1'
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
qc_channel_names()

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Hepatocyte',
        'strategy': [{'channel': 'Hep-Par-1', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'E-cadherin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Treg',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'FOXP3', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD8 T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Dendritic',
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [{'channel': 'CD68', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage - M2',
        'strategy': [{'channel': 'CD68', 'value': val, 'direction': 'pos'},
            {'channel': 'CD163', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD163', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### HT397B1

In [ ]:
case = 'HT397B1'
runs = [
    '2023-03-15_HT397B1',
    '2023-03-16_HT397B1',
    '2023-04-07_HT397B1',
    '2023-04-13_HT397B1'
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
qc_channel_names()

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'E-cadherin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Treg',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'FOXP3', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'}, # no cd3 in some of these sections
            {'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Dendritic', # not all have cd11b
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Macrophage',
        'strategy': [{'channel': 'CD68', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage - M2',
        'strategy': [{'channel': 'CD68', 'value': val, 'direction': 'pos'},
            {'channel': 'CD163', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD163', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### S18-9906

In [ ]:
case = 'S18-9906'
runs = [
    '2023-10-06_S18-9906'
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
qc_channel_names()

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'E-cadherin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Treg',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'FOXP3', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [{'channel': 'CD68', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage - M2',
        'strategy': [{'channel': 'CD68', 'value': val, 'direction': 'pos'},
            {'channel': 'CD163', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD163', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### S18-25943-A7

In [ ]:
case = 'S18-25943-A7'
runs = [
    '2023-11-22__S18-25943'
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
qc_channel_names()

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Islet',
        'strategy': [{'channel': 'INS', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'E-cadherin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Treg',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'FOXP3', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [{'channel': 'CD68', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage - M2',
        'strategy': [{'channel': 'CD68', 'value': val, 'direction': 'pos'},
            {'channel': 'CD163', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD163', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### S18-5591-C8

In [ ]:
case = 'S18-5591-C8'
runs = [
    '2023-11-16__S18-5591',
    '2023-11-18__S18-5591'
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
qc_channel_names()

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'E-cadherin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Treg',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'FOXP3', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
            {'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}
        ]},
    {'name': 'Macrophage - M2',
        'strategy': [{'channel': 'CD68', 'value': val, 'direction': 'pos'},
            {'channel': 'CD163', 'value': val, 'direction': 'pos'},
            {'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD163', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### HT206B1

In [ ]:
case = 'HT206B1'
runs = [
    '2023-09-14_HT206B1',
    '2023-10-02_HT206B1'
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
qc_channel_names()

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
val = .05
gating_strategy = [ # cd3 not present in all sections
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'E-cadherin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Treg',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'FOXP3', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Macrophage - M2',
        'strategy': [{'channel': 'CD68', 'value': val, 'direction': 'pos'},
            {'channel': 'CD163', 'value': val, 'direction': 'pos'},
            {'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
#             {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD163', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### C3L-00982

In [ ]:
case = 'C3L-00982'
runs = [
    '2024-01-17_C3L-00982',
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
case = 'C3L-00982'
runs = [
    '2024-01-17_C3L-00982',
]

In [ ]:
qc_channel_names()

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Proximal Tubule',
        'strategy': [{'channel': 'AQP1', 'value': val, 'direction': 'pos'},
                    {'channel': 'CD31', 'value': val, 'direction': 'neg'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'CA9', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'LRP2', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage - CD68+/IBA1+',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
            {'channel': 'IBA1', 'value': val, 'direction': 'pos'}
        ]},
    {'name': 'Macrophage - IBA1+',
        'strategy': [
            {'channel': 'IBA1', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Macrophage - CD68+',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### C3L-02551

In [ ]:
case = 'C3L-02551'
runs = [
    '2024-01-18_C3L-02551',
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
qc_channel_names()

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Proximal Tubule',
        'strategy': [{'channel': 'AQP1', 'value': val, 'direction': 'pos'},
                    {'channel': 'CD31', 'value': val, 'direction': 'neg'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'CA9', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'LRP2', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage - CD68+/IBA1+',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
            {'channel': 'IBA1', 'value': val, 'direction': 'pos'}
        ]},
    {'name': 'Macrophage - IBA1+',
        'strategy': [
            {'channel': 'IBA1', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Macrophage - CD68+',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### C3L-00970

In [ ]:
case = 'C3L-00970'
runs = [
    '2023-12-20_C3L-00970__C3N-00733__PA00002352',
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]
metadata = metadata[[True if case in x else False for x in metadata['specimen']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
qc_channel_names()

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Proximal Tubule',
        'strategy': [{'channel': 'AQP1', 'value': val, 'direction': 'pos'},
                    {'channel': 'CD31', 'value': val, 'direction': 'neg'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'CA9', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'LRP2', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### C3L-01287

In [ ]:
case = 'C3L-01287'
runs = [
    '2023-12-19_C3L-01287__C3N-01200',
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]
metadata = metadata[[True if case in x else False for x in metadata['specimen']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
qc_channel_names()

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Proximal Tubule',
        'strategy': [{'channel': 'AQP1', 'value': val, 'direction': 'pos'},
                    {'channel': 'CD31', 'value': val, 'direction': 'neg'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'CA9', 'value': val, 'direction': 'pos'}]},
    {'name': 'Epithelial',
        'strategy': [{'channel': 'LRP2', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### HT413C1-K2

In [ ]:
case = 'HT413C1-K2'
runs = [
    '2023-04-27_HT339B2__HT413C1__HT553P1__HT565B1',
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]
metadata = metadata[[True if case in x else False for x in metadata['specimen']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
qc_channel_names()

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### HT339B2-H1

In [ ]:
case = 'HT339B2-H1'
runs = [
    '2023-04-27_HT339B2__HT413C1__HT553P1__HT565B1',
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]
metadata = metadata[[True if case in x else False for x in metadata['specimen']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
qc_channel_names()

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### HT553P1-H2

In [ ]:
case = 'HT553P1-H2'
runs = [
    '2023-04-27_HT339B2__HT413C1__HT553P1__HT565B1',
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]
metadata = metadata[[True if case in x else False for x in metadata['specimen']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
qc_channel_names()

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### HT565B1-H2

In [ ]:
case = 'HT565B1-H2'
runs = [
    '2023-04-27_HT339B2__HT413C1__HT553P1__HT565B1',
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]
metadata = metadata[[True if case in x else False for x in metadata['specimen']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
qc_channel_names()

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

###### HT448C1-Th1K1Fp1

In [ ]:
case = 'HT448C1-Th1K1Fp1'
runs = [
    '2023-04-13_HT488C1',
    '2023-04-19_HT488C1'
]

metadata = pd.read_csv(specimen_metadata_fp, sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]
# metadata = metadata[[True if case in x else False for x in metadata['specimen']]]

output_dir = os.path.join(parent_dir, case)
spatial_dir = os.path.join(output_dir, 'spatial_features')
Path(spatial_dir).mkdir(parents=True, exist_ok=True)

metadata

In [ ]:
qc_channel_names()

In [ ]:
channel_thresholds = get_channel_thresholds()
channel_thresholds.keys(), next(iter(channel_thresholds.values()))

In [ ]:
val = .05
gating_strategy = [
    {'name': 'Epithelial',
        'strategy': [{'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'pos'}]},
    {'name': 'CD4 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD4', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'CD8 T cell',
        'strategy': [
            {'channel': 'CD3e', 'value': val, 'direction': 'pos'},
            {'channel': 'CD8', 'value': val, 'direction': 'pos'}]},
    {'name': 'T cell',
        'strategy': [{'channel': 'CD3e', 'value': val, 'direction': 'pos'}]},
    {'name': 'Macrophage',
        'strategy': [
            {'channel': 'CD68', 'value': val, 'direction': 'pos'},
        ]},
    {'name': 'Dendritic', 
        'strategy': [
            {'channel': 'CD11b', 'value': val, 'direction': 'pos'},
            {'channel': 'CD3e', 'value': val, 'direction': 'neg'}
        ]},
    {'name': 'B cell',
        'strategy': [
            {'channel': 'CD20', 'value': val, 'direction': 'pos'},
            {'channel': 'Pan-Cytokeratin', 'value': val, 'direction': 'neg'},
            {'channel': 'E-cadherin', 'value': val, 'direction': 'neg'}]},
    {'name': 'Endothelial',
        'strategy': [{'channel': 'CD31', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD45', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD11b', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'CD4', 'value': val, 'direction': 'pos'}]},
    {'name': 'Immune',
        'strategy': [{'channel': 'HLA-DR', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'SMA', 'value': val, 'direction': 'pos'}]},
    {'name': 'Fibroblast',
        'strategy': [{'channel': 'Podoplanin', 'value': val, 'direction': 'pos'}]}
]

## gate cells

In [ ]:
set(specimen_to_ome.keys()), set(specimen_to_seg.keys())

In [ ]:
fps = sorted(utils.listfiles(output_dir, f'cell_segmentation.tif$'))
specimen_to_ome = {specimen:fp for specimen, fp in zip(metadata['specimen'], metadata['filepath'])}
specimen_to_seg = {fp.split('/')[-1].replace('_cell_segmentation.tif', ''):fp for fp in fps}
set(specimen_to_ome.keys()) - set(specimen_to_seg.keys())

In [ ]:
assert sorted([fp.split('/')[-1] for fp in specimen_to_ome.values()]) == sorted(channel_thresholds.keys())

In [ ]:
# sorted([fp.split('/')[-1] for fp in specimen_to_ome.values()]), sorted(channel_thresholds.keys())

In [ ]:
for specimen, ome_fp in specimen_to_ome.items():
    print(specimen)
    seg_fp = specimen_to_seg[specimen]
    thresholds = channel_thresholds[ome_fp.split('/')[-1]]
    
    df, a = mip.get_spatial_features(seg_fp, ome_fp, thresholds=thresholds, gating_strategy=gating_strategy)
    a.write_h5ad(os.path.join(spatial_dir, f'{specimen}_spatial_features.h5ad'))
    df.to_csv(os.path.join(spatial_dir, f'{specimen}_spatial_features.txt'), sep='\t', index=False)

In [ ]:
fps = sorted(utils.listfiles(spatial_dir, regex=r'_spatial_features.txt'))
fps

In [ ]:
df = pd.read_csv(next(iter(fps)), sep='\t')
df

In [ ]:
cell_types = sorted(set([x['name'] for x in gating_strategy]))
cell_types.append('Unlabeled')
cmap = sns.color_palette('tab20') + sns.color_palette('tab20b') + sns.color_palette('tab20c')
cell_type_to_color = {ct:c for ct, c in zip(cell_types, cmap)}

In [ ]:
def format_axis(ax):
    ax.invert_yaxis()
    ax.axis('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.title.set_fontsize(8)
    ax.yaxis.label.set_fontsize(6)

nrows, ncols = len(fps), len(cell_types) + 1
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols, nrows))
if nrows == 1:
    axs = rearrange(axs, 'n -> 1 n')
for i, fp in enumerate(fps):
    ax = axs[i, 0]
    sid = fp.split('/')[-1].replace('_spatial_features.txt', '')
    df = pd.read_csv(fp, sep='\t')
    ax.scatter(df['col'], df['row'], c=[cell_type_to_color[x] for x in df['cell_type']], s=.02, marker='.')
    ax.set_ylabel(sid, rotation=0)
    if i == 0:
        ax.set_title('cell_type', rotation=90)
    format_axis(ax)
    
    for j, ct in enumerate(cell_types):
        ax = axs[i, j + 1]
        f = df[df['cell_type']==ct]
        ax.scatter(f['col'], f['row'], s=.02, marker='.')
        
        if i == 0:
            ax.set_title(ct, rotation=90)
        format_axis(ax)

plt.savefig(os.path.join(spatial_dir, f'cell_types.png'))

In [ ]:
data, sids = [], []
for i, fp in enumerate(fps):
    sid = fp.split('/')[-1].replace('_spatial_features.txt', '')
    df = pd.read_csv(fp, sep='\t')
    counts = Counter(df['cell_type'])
    data.append([counts.get(ct, 0) for ct in cell_types])
    sids.append(sid)
count_df = pd.DataFrame(data=data, columns=cell_types, index=sids)
count_df

In [ ]:
frac_df = count_df.copy()
frac_df /= count_df.values.sum(1, keepdims=True)
frac_df